__качаем объявления о продаже недвижимости с avito.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
import logging
import sys 

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

In [2]:
from time import sleep
from tqdm.auto import tqdm
import pandas as pd
from datetime import datetime 

pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

In [3]:
source = [
    {
            'place': 'sevastopol',
        'subfolder': 'flat',
              'url': 'https://www.avito.ru/sevastopol/kvartiry/prodam',
    },{
            'place': 'sevastopol',
        'subfolder': 'land',
              'url': 'https://www.avito.ru/sevastopol/zemelnye_uchastki/prodam',
    },{
            'place': 'sevastopol',
        'subfolder': 'house',
              'url': 'https://www.avito.ru/sevastopol/doma_dachi_kottedzhi/prodam',
    },
    {
            'place': 'sudak',
        'subfolder': 'flat',
              'url': 'https://www.avito.ru/sudak/kvartiry/prodam',
    },
    {
            'place': 'sudak',
        'subfolder': 'land',
              'url': 'https://www.avito.ru/sudak/zemelnye_uchastki/prodam',
    },{
            'place': 'sudak',
        'subfolder': 'house',
              'url': 'https://www.avito.ru/sudak/doma_dachi_kottedzhi/prodam',
    },
#     {
#             'place': 'bahchisaray',
#         'subfolder': 'flat',
#               'url': 'https://www.avito.ru/bahchisaray/kvartiry/prodam',
#     }, 
    {
            'place': 'bahchisaray',
        'subfolder': 'land',
              'url': 'https://www.avito.ru/bahchisaray/zemelnye_uchastki/prodam',
    },{
            'place': 'bahchisaray',
        'subfolder': 'house',
              'url': 'https://www.avito.ru/bahchisaray/doma_dachi_kottedzhi/prodam',
    },

]

---

In [4]:
# from selenium import webdriver
# from lib.avito import AvitoPageParser

# browser = webdriver.Firefox()
# browser.get(source[0]['url'])
# data = AvitoPageParser().transform(browser)
# # browser.close()

In [5]:
# browser.close()

In [6]:
# data

---

In [7]:
from selenium import webdriver
# from selenium.webdriver.common.by import By
from lib.avito import AvitoParser

In [8]:
dest_folder = 'data/2024/raw'

In [9]:
%%time 


options = webdriver.FirefoxOptions()
#options.headless = True
browser = webdriver.Firefox(options=options)
logging.info('open virtual browser')

for s in source:

    url = s['url']
    place = s['place']
    subfolder = s['subfolder'] 
    
    display(url)
    browser.get(url)
    sleep(3)
    
    data = AvitoParser().transform(browser)
    assert len(data)>0
    
    df = pd.DataFrame(data)
    df['ts'] = datetime.now()
    df['place'] = place

    display( len(df) )
    t_now = datetime.now().strftime('%Y-%m-%d_%H-%M')
    fname = f'{dest_folder}/{subfolder}/avito/avito_{t_now}_raw_{place}.pkl'
    df.to_pickle(fname)
    display( fname )

browser.close()

[INFO    ] 2024-07-16 12:24:08 | open virtual browser


'https://www.avito.ru/sevastopol/kvartiry/prodam'

[INFO    ] 2024-07-16 12:24:31 | page 1: 53 items
[INFO    ] 2024-07-16 12:24:44 | page 2: 50 items
[INFO    ] 2024-07-16 12:24:55 | page 3: 50 items
[INFO    ] 2024-07-16 12:25:06 | page 4: 50 items
[INFO    ] 2024-07-16 12:25:15 | page 5: 50 items
[INFO    ] 2024-07-16 12:25:25 | page 6: 50 items
[INFO    ] 2024-07-16 12:25:34 | page 7: 50 items
[INFO    ] 2024-07-16 12:25:44 | page 8: 50 items
[INFO    ] 2024-07-16 12:25:55 | page 9: 50 items
[INFO    ] 2024-07-16 12:26:05 | page 10: 50 items
[INFO    ] 2024-07-16 12:26:15 | page 11: 50 items
[INFO    ] 2024-07-16 12:26:25 | page 12: 50 items
[INFO    ] 2024-07-16 12:26:36 | page 13: 50 items
[INFO    ] 2024-07-16 12:26:47 | page 14: 50 items
[INFO    ] 2024-07-16 12:27:03 | page 15: 50 items
[INFO    ] 2024-07-16 12:27:16 | page 16: 50 items
[INFO    ] 2024-07-16 12:27:27 | page 17: 50 items
[INFO    ] 2024-07-16 12:27:38 | page 18: 50 items
[INFO    ] 2024-07-16 12:27:49 | page 19: 50 items
[INFO    ] 2024-07-16 12:28:00 | page 20

1448

'data/2024/raw/flat/avito/avito_2024-07-16_12-34_raw_sevastopol.pkl'

...

'https://www.avito.ru/bahchisaray/doma_dachi_kottedzhi/prodam'

[INFO    ] 2024-07-16 12:52:32 | page 1: 50 items
[INFO    ] 2024-07-16 12:52:42 | page 2: 50 items
[INFO    ] 2024-07-16 12:52:50 | page 3: 50 items
[INFO    ] 2024-07-16 12:52:58 | page 4: 50 items
[INFO    ] 2024-07-16 12:53:07 | page 5: 50 items
[INFO    ] 2024-07-16 12:53:15 | page 6: 40 items


290

'data/2024/raw/house/avito/avito_2024-07-16_12-53_raw_bahchisaray.pkl'

CPU times: user 3min 5s, sys: 7.32 s, total: 3min 12s
Wall time: 29min 8s


In [10]:
# browser.close()

In [11]:
!date

Вт 16 июл 2024 12:53:15 MSK
